In [0]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
keras = tf.keras

In [0]:
tf.__version__

'1.12.0'

In [0]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
X_train = X_train[..., None]
X_test = X_test[..., None]

In [0]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 28, 28, 1), (60000,), (10000, 28, 28, 1), (10000,))

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
X_train = X_train / 255
X_test = X_test / 255

In [0]:
number_of_classes = 10

Y_train = keras.utils.to_categorical(Y_train, number_of_classes)
Y_test = keras.utils.to_categorical(Y_test, number_of_classes)

Y_train[0], Y_test[0]

(array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32))

In [0]:
class DenseModel(keras.Model):
  def __init__(self):
    super(DenseModel, self).__init__()
    self.dense1 = keras.layers.Dense(128, activation='relu')
    self.dense2 = keras.layers.Dense(10)
    self.flatten = keras.layers.Flatten()
    
  @tf.contrib.eager.defun()
  def call(self, x, training):
    x = self.flatten(x)
    
    x = self.dense1(x)
    x = self.dense2(x)
    
    return x

In [0]:
class ConvModel(keras.Model):
  def __init__(self):
    super(ConvModel, self).__init__()
    self.conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu')
    self.conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu')
    self.maxpool = keras.layers.MaxPool2D()
    self.dense1 = keras.layers.Dense(128, activation='relu')
    self.dense2 = keras.layers.Dense(10)
    self.flatten = keras.layers.Flatten()
    
  @tf.contrib.eager.defun()
  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = self.maxpool(x)
    
    x = self.flatten(x)
    
    x = self.dense1(x)
    x = self.dense2(x)
    
    return x

In [0]:
model = ConvModel()

In [0]:
BATCH_SIZE = 64

In [0]:
train = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(len(X_train)).batch(BATCH_SIZE)

In [0]:
test = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(BATCH_SIZE)

# Custom training loop

In [0]:
optimizer = tf.train.AdamOptimizer(1e-3)

In [0]:
def loss_function(y, pred):
  return tf.losses.softmax_cross_entropy(y, pred)

In [0]:
for epoch in range(2):
  start = time.time()
    
  for (x, y) in train:
    with tf.GradientTape() as tape:
      # get the predictions
      predictions = model(x)
      # calculate the loss
      loss = loss_function(y, predictions)
      
    # calculate the gradients of loss wrt input
    gradients = tape.gradient(loss, model.variables)
    # backpropagate and take the step in the direction of the gradient
    optimizer.apply_gradients(zip(gradients, model.variables))
  
  print ('Training Set: Epoch {:04d}, Loss = {:.3f}'.format(epoch+1, loss))
  print ('Time taken {}'.format(time.time() - start))
  
#   # Validation Step
#   for xt, yt in test:
#     preds_test = model(xt)

#   print ('Validation Set: Loss = {:.3f}\n'.format(loss))

Training Set: Epoch 0001, Loss = 0.020
Time taken 17.43385624885559
Training Set: Epoch 0002, Loss = 0.054
Time taken 17.41017198562622


In [0]:
s = time.time()
c = 1
for x, y in train:
  print (y)
print (time.time()-s)